# Benchmark Models

## Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm.auto import tqdm
import pathlib
import datetime
import subprocess
import numpy as np
import pandas as pd
import lifelines

In [ ]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

experiment = '231117'
experiment_path = f"{project_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(10)]

today = experiment 

In [ ]:
endpoint_defs = pd.read_feather(f"{base_path}/data/endpoints_epic_md.feather").set_index("endpoint")
endpoints = endpoint_defs.index.to_list()

In [ ]:
from scripts.benchmarks_iteration import load_data, calculate_iteration

In [ ]:
os.environ['MKL_NUM_THREADS'] = "1"
os.environ['NUMEXPR_NUM_THREADS'] = "1"
os.environ['OMP_NUM_THREADS'] = "1"

In [ ]:
rows_ray = []
iterations = [i for i in range(100)]
model = "RetinaUKB"
t_eval = 10
for iteration in tqdm(iterations[:10]):
    output_path, experiment_path, in_path, out_path, endpoints, scores, prediction_paths, eids_dict = load_data()
    for endpoint in tqdm(endpoints): 
        eids_e = eids_dict[endpoint]

        #ds = calculate_iteration.remote(in_path, prediction_paths, endpoint, scores, partition, model, t_eval, iteration, eids_e, output_path) #ray

        ds = calculate_iteration(in_path, prediction_paths, endpoint, scores, model, t_eval, iteration, eids_e, output_path) #ray
        rows_ray.append(ds)

        del eids_e

In [ ]:
#rows = [ray.get(r) for r in tqdm(rows_ray)] # ray
rows = rows_ray # not ray
rows_finished = [item for sublist in rows for item in sublist]
benchmark_endpoints = pd.DataFrame(rows_finished)

benchmark_endpoints.to_feather(f"{experiment_path}/benchmarks_231117.feather")

In [ ]:
benchmark_endpoints.groupby("score")["cindex"].mean().to_frame("cindex").sort_values("cindex")

In [ ]:
benchmark_endpoints.groupby("score")["cindex"].mean().to_frame("cindex").sort_values("cindex")

In [ ]:
benchmark_endpoints.groupby("score")["cindex"].mean().to_frame("cindex").sort_values("cindex")